# Анализ сервиса электронных книг (SQL)

<div style="border:solid green 2px; padding: 20px">

**Компания заказчик** - Стартап по предоставлению электронных книг по подписке

**Основная цель исследования** - Проанализировать базу данных только что приобретенного крупного сервиса для чтения книг по подписке  с целью формирования целостного предложения для нового продукта.

**Задачи исследования:**
- проанализировать информацию о книгах, издательствах, авторах
- проанализировать пользовательские обзоры книг

---

**Описание данных**
_**Таблица `books`**_ - содержит информацию о книгах

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

_**Таблица `authors`**_ - содержит информацию об авторах

- `author_id` — идентификатор автора;
- `author` — имя автора.

_**Таблица `publishers`**_ - содержит информацию об издательствах

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

_**Таблица `ratings`**_ - содержит информацию о пользовательских оценках

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

_**Таблица `reviews`**_ - содержит информацию о пользовательских обзорах

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подготовительный этап

In [1]:
# установка необходимой библиотеки
!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Выведем таблицы для знакомства.

In [3]:
# таблица с информацией о книгах
query = '''
        SELECT * 
        FROM books;
'''.format('books')
    

books = pd.io.sql.read_sql(query, con = engine) 
books.head(1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [4]:
# таблица с информацией об авторах
query = '''
        SELECT * 
        FROM authors;
'''.format('authors')
    

authors = pd.io.sql.read_sql(query, con = engine) 
authors.head(1)

,author_id,author
0,1,A.S. Byatt


In [5]:
# таблица с информацией об издательствах
query = '''
        SELECT * 
        FROM publishers;
'''.format('publishers')
    

publishers = pd.io.sql.read_sql(query, con = engine) 
publishers.head(1)

,publisher_id,publisher
0,1,Ace


In [6]:
# таблица с информацией об оценках книг
query = '''
        SELECT * 
        FROM ratings;
'''.format('ratings')
    

ratings = pd.io.sql.read_sql(query, con = engine) 
ratings.head(1)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [7]:
# таблица с информацией об обзорах
query = '''
        SELECT * 
        FROM reviews;
'''.format('reviews')
    

reviews = pd.io.sql.read_sql(query, con = engine) 
reviews.head(1)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## Задача 1

<div style="border:solid black 0.5px; padding: 20px">

Необходимо подсчитать, сколько книг вышло после 1 января 2000 года.

In [8]:
task_1 = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01';
'''.format('task_1')
    

task_1 = pd.io.sql.read_sql(task_1, con = engine) 
task_1

,count
0,819


### Вывод

- После 1 января 2000 года вышло 819 книг

## Задача 2

<div style="border:solid black 0.5px; padding: 20px">

Необходимо расчитать для каждой книги:
- количество обзоров
- среднюю оценку

In [9]:
task_2 = '''
        SELECT b.book_id,
               b.title, 
               COUNT(DISTINCT rev.review_id) AS count_review,  
               AVG(rat.rating) AS avg_ratings         
        FROM books AS b
        LEFT JOIN ratings AS rat ON rat.book_id = b.book_id 
        LEFT JOIN reviews AS rev ON rev.book_id = b.book_id 
        GROUP BY b.book_id
        ORDER BY count_review DESC, avg_ratings DESC;
'''.format('task_2')
    

task_2 = pd.io.sql.read_sql(task_2, con = engine) 
task_2

,book_id,title,count_review,avg_ratings
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


### Вывод

- Первое место по числу обзоров занимает первая книга в саге Сумерки (Twilight (Twilight #1)), что абсолютно не удивительно, учитывая популярность данного издания в свое время. Средняя оценка 3,66, что далеко не самый лучший результат оценки читателем.
- Одинаковое число обзоров (6) у нескольких произведений:
>-  Гарри Поттер и Узник Азкабана (средняя оценка 4,41)
>-  Гарри Поттер и Тайная Комната (средняя оценка 4,28)
>-  Воровка книг (средняя оценка 4,26)
>-  Замок из стекла (средняя оценка 4,2)

Проверим какая книга занимает первое место по средней оценке с учетом числа обзоров

In [10]:
task_2_2 = '''
        SELECT b.book_id,
               b.title, 
               COUNT(DISTINCT rev.review_id) AS count_review,  
               AVG(rat.rating) AS avg_ratings         
        FROM books AS b
        LEFT JOIN ratings AS rat ON rat.book_id = b.book_id 
        LEFT JOIN reviews AS rev ON rev.book_id = b.book_id 
        GROUP BY b.book_id
        ORDER BY avg_ratings DESC, count_review DESC
        LIMIT 1;
'''.format('task_2')
    

task_2_2 = pd.io.sql.read_sql(task_2_2, con = engine) 
task_2_2

,book_id,title,count_review,avg_ratings
0,17,A Dirty Job (Grim Reaper #1),4,5.0


- Первое место по числу обзоров, и с максимальной оценкой - 5 - занимает произведение Кристофера Мура "Грязна работа"

## Задача 3

<div style="border:solid black 0.5px; padding: 20px">

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (в целях исключения из анализа брошюр)

In [11]:
task_3 = '''
        SELECT p.publisher,
               COUNT(b.book_id) AS count_books
        FROM publishers AS p
        LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE num_pages > 50
        GROUP BY p.publisher
        ORDER BY count_books DESC
        LIMIT 1;
'''.format('task_3')
    

task_3 = pd.io.sql.read_sql(task_3, con = engine) 
task_3

,publisher,count_books
0,Penguin Books,42


### Вывод

- Первое место среди издательств по числу выпущенных книг (исключая брошюры толщиной менее 50 страниц), занимает издательство "Penguin Books"

## Задача 4

<div style="border:solid black 0.5px; padding: 20px">

Определить автора с самой высокой средней оценкой книг (необходимо учитывать только книги с 50 и более оценками)

In [12]:
task_4 = '''
        SELECT author,
               AVG(second.rating) AS avg_rating
        FROM (
                SELECT author,
                       AVG(rating) AS rating
                FROM (
                        SELECT *
                        FROM books
                        WHERE book_id IN (SELECT book_id  
                                          FROM ratings
                                          GROUP BY book_id
                                          HAVING COUNT(rating) >= 50)) AS first
                                         
                INNER JOIN authors AS a ON first.author_id = a.author_id
                INNER JOIN ratings AS rat ON first.book_id = rat.book_id
                GROUP BY author
                ORDER BY author) AS second
                
        GROUP BY author
        ORDER BY avg_rating DESC
        LIMIT 1
'''.format('task_4')
    

task_4 = pd.io.sql.read_sql(task_4, con = engine) 
task_4

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


### Вывод

- Автором с самой высокой средней оценкой книг (на основании более 50 оценок), является несравненная Джоан Роулинг. Речь идет о книгах проиллюстрированных Мэри ГрандПре (американский иллюстратор и писатель) - средняя оценка произведений 4,28. 



## Задача 5

<div style="border:solid black 0.5px; padding: 20px">

Расчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
task_5 = '''
        SELECT ROUND (AVG(second.count_reviewer)) AS avg_reviews
        FROM
            (SELECT COUNT(review_id) AS count_reviewer
             FROM (
                    SELECT r.username
                    FROM ratings AS r
                    GROUP BY r.username
                    HAVING COUNT(rating_id) > 50
            ) AS first
            
        LEFT JOIN reviews AS rev ON first.username = rev.username
        GROUP BY rev.username) AS second
'''.format('task_5')
    

task_5 = pd.io.sql.read_sql(task_5, con = engine) 
task_5

,avg_reviews
0,24.0


### Вывод

- В среднем, пользователь, который выставил более 50 оценок, оставляет около 24 обзоров. 

## Общий вывод по исследованию

<div style="border:solid green 2px; padding: 20px">
    
В результате проведенного иссредования базы данных, приобретенного сервиса предоставления электронных книг по подписке, были сделаны следующие выводы:
- После 1 января 2000 года вышло 819 книг
- Первое место по числу обзоров занимает первая книга в саге Сумерки (Twilight (Twilight #1)), что абсолютно не удивительно, учитывая популярность данного издания в свое время. На нее оставлено 7 обзоров. Средняя оценка произведения - 3,66.
Одинаковое число обзоров (6) у нескольких произведений:
>-  Гарри Поттер и Узник Азкабана (средняя оценка 4,41)
>-  Гарри Поттер и Тайная Комната (средняя оценка 4,28)
>-  Воровка книг (средняя оценка 4,26)
>-  Замок из стекла (средняя оценка 4,2)
- Первое место по числу обзоров, имея максимальную среднюю оценку (5 из 5 возможных) занимает произведение Кристофера Мура "Грязна работа"
- Первое место среди издательств по числу выпущенных книг (исключая брошюры толщиной менее 50 страниц), занимает издательство "Penguin Books"
- Автором с самой высокой средней оценкой книг (на основании более 50 оценок), является несравненная Джоан Роулинг. Речь идет о книгах проиллюстрированных Мэри ГрандПре (американский иллюстратор и писатель) - средняя оценка произведений 4,28. 
- В среднем, пользователь, который выставил более 50 оценок, оставляет около 24 обзоров. 